<a href="https://colab.research.google.com/github/MasslessAI/narratelab/blob/master/exp/exp_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [140]:
from google.colab import drive

drive.mount('gdrive', force_remount=True)

root_dir = '/content/gdrive/'

gdrive_path = root_dir + 'MyDrive/narratelab/exp_1'

!pip install spacy keybert[spacy] sentence-transformers redditcleaner psaw pandas loguru bertopic[spacy] distinctipy colour
# !python -m spacy download en_core_web_trf

Mounted at gdrive


# Scrap Reddit Data

Using pushshift api to quickly search for submissions where title contains **question-indicative phrases/words**. Only retrieve submissions whose **num_comments > 1**, this filters out most of the ads.

refer to [Content Ideas From Reddit](https://timothywangdev.github.io/knowledge/Content%20Marketing/content-ideas-from-reddit)

In [193]:
import datetime as dt
from datetime import date, datetime
from loguru import logger
import pandas as pd
from psaw import PushshiftAPI
import time
import os
import redditcleaner
import re
import base64
import IPython
from distinctipy import distinctipy
import random
from colour import Color

api = PushshiftAPI()

QUESTION_WORDS = [
  "what",
  "when",
  "where",
  "who",
  "whom",
  "which",
  "whose",
  "why",
  "how",
  "wonder",
  "want",
  "is anyone",
  "does anyone",
  "any tips",
  "advice",
  "suggestion",
  "suggestions",
  "suggest",
  "ideas on",
  "need help",
  "needs help",
  "need your help",
  "serious help",
  "please help",
  "challenge",
  "challenges",
  "can't stand",
  "struggle",
  "struggling",
  "can't figure out",
  "help me",
  "hardest part",
  "would appreciate",
  "would really appreciate",
  "any guidance",
  "no idea",
  "confused with",
  "new to",
  "is there any way"
]

# (Optional) Do some experiments on question words and psaw filters

In [192]:

# do some exps on question words and psaw filters
_QUESTION_WORDS=["ideas on", "struggle with", "suggestion"]
title_query = '|'.join(map(lambda x: '"{}"'.format(x), _QUESTION_WORDS))
print(title_query)

gen = list(
        api.search_submissions(
            title=title_query,
            is_self=True,
            is_original_content=True, 
            subreddit='content_marketing',
            num_comments=">1",
            filter=['title', 'selftext', 'author', 'permalink', 'num_comments', 'score', 'total_awards_received',
                    'upvote_ratio'],
            sort='asc', sort_type='created_utc', limit=500))

titles = [item.d_['title'] for item in gen]

# generate distinct hsl colors
bg_color_list = distinctipy.get_colors(50)
inverted_color_list = distinctipy.invert_colors(bg_color_list)
text_color_list = [distinctipy.get_text_color(color) for color in bg_color_list]

# convert colors to hex
bg_color_list = [Color(rgb=color).hex for color in bg_color_list]
inverted_color_list = [Color(rgb=color).hex for color in inverted_color_list]
text_color_list = [Color(rgb=color).hex for color in text_color_list]

def colored(word, color_idx=0):
  color = bg_color_list[color_idx]
  return '<span style="display: inline-block; font-weight: 600; font-size: 16px; border-radius: 15%; padding-left: 5px; padding-right: 5px; margin: 4px 2px; background-color: {}; color:{};">{}</span>'.format(bg_color_list[color_idx], text_color_list[color_idx], word)
  
colored_titles = []
for title in titles:
  _title = title.lower()
  for idx, question_word in enumerate(_QUESTION_WORDS):
    _title = _title.replace(question_word, colored(question_word, color_idx = idx))
  colored_titles.append('<li>' + _title + "</li>")

IPython.display.HTML("<ol>" +' '.join(colored_titles) + "</ol>")

"ideas on"|"struggle with"|"suggestion"


Only retrieve submissions that are text submissions, and filter out deleted submissions and submissions whose author is banned.

We keep the following fields:

* title
* selftext
* author
* permalink
* num_comments <font color='blue'>*</font>
* score <font color='blue'>*</font>
* upvote_ratio <font color='blue'>*</font>
* total_awards_received <font color='blue'>*</font>

<font color='blue'>*</font> *Used for future ranking purpose*

In [194]:
start_epoch = int(dt.datetime(2019, 1, 1).timestamp())
#end_epoch = int(dt.datetime(2020, 3, 1).timestamp())
end_epoch = int(time.time())
total = 0

SUBREDDIT = 'nft'
DATA_FILE_NAME = gdrive_path + '/reddit_submission_{}_{}_{}.tsv'.format(
    SUBREDDIT, datetime.fromtimestamp(start_epoch).strftime("%Y_%m_%d"),
    datetime.fromtimestamp(end_epoch).strftime("%Y_%m_%d"))

title_query = '|'.join(map(lambda x: '"{}"'.format(x), QUESTION_WORDS))
while True:
    gen = list(
        api.search_submissions(
            after=start_epoch, 
            before=end_epoch,
            title=title_query,
            is_self=True,
            is_original_content=True,
            subreddit=SUBREDDIT,
            num_comments=">1",
            filter=['title', 'selftext', 'author', 'permalink', 'num_comments', 'score', 'total_awards_received',
                    'upvote_ratio'],
            sort='asc', sort_type='created_utc', limit=500))

    if len(gen) == 0:
        break

    def submission_filter(submission):
        if 'title' not in submission:
            return False
        if 'selftext' not in submission:
            return False
        if 'author' not in submission:
            return False
        if submission['author'] == "[deleted]":
            return False
        if any(submission['selftext'] == x for x in ["[removed]", "[deleted]"]):
            return False
        return True

    def prepare_data(data):
        # some of the fields may be missing
        # must manually set an init value to avoid
        # generating invalid csv
        _data = {
            'title': '',
            'selftext': '',
            'author': '',
            'permalink': '',
            'num_comments': 0,
            'score': 0,
            'total_awards_received': 0,
            'upvote_ratio': 1.0,
            'created_utc': None
        }

        for key in _data:
            if key in data and data[key] is not None:
                _data[key] = data[key]

        return _data


    items = map(prepare_data, [item.d_ for item in gen])

    items = list(filter(submission_filter, items))

    df = pd.DataFrame(items)

    # clean data
    def clean(text):
        # remove reddit styles
        text = redditcleaner.clean(
            text, quote=False, bullet_point=False, link=False, strikethrough=False, spoiler=False, code=False,
            superscript=False, table=False)

        # refer to https://towardsdatascience.com/cleaning-text-data-with-python-b69b47b97b76
        # Remove unicode characters
        text = text.encode('ascii', 'ignore').decode()

        # Remove Hashtags
        text = re.sub("#\S+", " ", text)

        # Remove markdown links
        text = re.sub(r"\[(.+)\]\(.+\)", r"\1", text)

        # Remove other urls
        text = re.sub(r"http\S+", " ", text)

        # remove text inside brackets
        text = re.sub("\(.*?\)"," ", text)
        text = re.sub("\[.*?\]"," ", text)

        # remove quotes
        # remove brackets
        # remove semicolon
        text = re.sub(r'[\t()[\]\"*:\\]',' ', text)

        # remove non-ascii chars
        text = re.sub(r"[^\x00-\x7F]+",' ', text)

         # Replace the over spaces
        text = re.sub('\s{2,}', " ", text)

        return text

    df['title'] = df['title'].map(clean)
    df['selftext'] = df['selftext'].map(clean)

    if not os.path.isfile(DATA_FILE_NAME):
        df.to_csv(DATA_FILE_NAME, sep='\t', header='column_names', index=False, quoting=3)
    else:  # else it exists so append without writing the header
        df.to_csv(DATA_FILE_NAME, sep='\t', mode='a', header=False, index=False, quoting=3)

    start_epoch = items[-1]['created_utc']
    total += len(items)
    logger.info('Added {} Total {} Last created_utc {}'.format(
        len(items), total, date.fromtimestamp(start_epoch)))

    time.sleep(3)

2021-07-14 23:56:20.108 | INFO     | __main__:<module>:119 - Added 355 Total 355 Last created_utc 2021-04-14
2021-07-14 23:56:26.071 | INFO     | __main__:<module>:119 - Added 308 Total 663 Last created_utc 2021-05-15
2021-07-14 23:56:34.763 | INFO     | __main__:<module>:119 - Added 320 Total 983 Last created_utc 2021-07-11
2021-07-14 23:56:38.393 | INFO     | __main__:<module>:119 - Added 10 Total 993 Last created_utc 2021-07-14


# Load Scrapped Reddit Data

Load submissions, filter out ads, and classified them by wh-words

From observation, some **submissions are ads**, which often contain the following symbols/phrases

```
'-', ':', ';' 
your business(es)
help you
case study
ALL CAPITAL chars
how i
dollar symbol $ (e.g turn $100 into 100k)
step-by-step
here is how
here's how
part 1/2/3
top [number] things
[number] reasons
ultimate guide
cheat sheet
cheatsheet
infographic
by [year]
ama
```

<mark>TODO</mark> 
 - [ ] Add more ad-indicative phrases
 - [ ] Use num_comments to filter ads

In [195]:
df = pd.read_csv(gdrive_path +"/reddit_submission_nft_2019_01_01_2021_07_12.tsv", sep='\t', quoting=3)

# data cleanup
print('before clean-up # rows: {}'.format(len(df)))
cleaned_rows = []

AD_INDICATIVE_PHRASES = [
  "your business", 
  "your businesses", 
  "help you", 
  "case study"
  "how i",
  "$",
  "step-by-step",
  "here is how",
  "here's how",
  "part 1",
  "part 2",
  "part 3",
  "ultimate guide",
  "cheatsheet",
  "infographic",
  "ama"
]

for index, row in df.iterrows():
    row_title = row['title'].lower()
    if not any(x in ['-', ':', ';'] for x in row_title) and not any(phrase in row_title for phrase in AD_INDICATIVE_PHRASES) and "?" in row_title:
        '''
        1. must contain '?'
        2. can only contain alphanumeric, punctuations and space
        3. should not contain '-', ':', ';' which indicates ads
        '''
        cleaned_rows.append(index)

df = df[df.index.isin(cleaned_rows)].reset_index()
print('after clean-up # rows: {}'.format(len(df)))

before clean-up # rows: 2812
after clean-up # rows: 1907


# Classified submissions by wh-words in title and aggregate stats

In [202]:
import pprint

title_cat = []
for index, row in df.iterrows():
    title_cat.append('NO_WH_WORD')
    for wh_word in QUESTION_WORDS:
        if wh_word in row['title'].lower():
            title_cat[-1] = wh_word
            break

df['title_cat'] = title_cat

cat_stats = {
    'cat': QUESTION_WORDS,
    'num_docs': [],
    'total_score': [],
    'total_comments': []
}

# print out number of docs per category
for wh_word in QUESTION_WORDS:
    _df = df[df['title_cat'] == wh_word]
    cat_stats['num_docs'].append(len(_df))
    cat_stats['total_score'].append(_df['score'].sum())
    cat_stats['total_comments'].append(_df['num_comments'].sum())
  
cat_stats_df = pd.DataFrame(data=cat_stats)

print(cat_stats_df.to_markdown())
  

|    | cat                     |   num_docs |   total_score |   total_comments |
|---:|:------------------------|-----------:|--------------:|-----------------:|
|  0 | what                    |        633 |           765 |             1825 |
|  1 | when                    |         58 |            58 |              224 |
|  2 | where                   |        114 |           146 |              420 |
|  3 | who                     |         72 |            78 |              122 |
|  4 | whom                    |          0 |             0 |                0 |
|  5 | which                   |        118 |           132 |              432 |
|  6 | whose                   |          0 |             0 |                0 |
|  7 | why                     |        120 |           130 |              542 |
|  8 | how                     |        590 |           658 |             1944 |
|  9 | wonder                  |          2 |             2 |                8 |
| 10 | want                 

# Pick a wh-word category

In [209]:
df_selection = df[df['title_cat'] == 'why'].copy().reset_index()
print(df_selection.head(10).to_markdown())

|    |   level_0 |   index | title                                                                                               | selftext                                                                                                                                                                                                                                                                                                                 | author             | permalink                                                                 |   num_comments |   score |   total_awards_received |   upvote_ratio |   created_utc | title_cat   |
|---:|----------:|--------:|:----------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Topic Modeling with BERTopic

In [218]:
from bertopic import BERTopic

topic_model = BERTopic(n_gram_range=(1, 2), nr_topics='auto', calculate_probabilities=True, min_topic_size=5)
topics, probabilities = topic_model.fit_transform(df_selection['title'])
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,74,-1_why_nft_nfts_why are
1,0,28,0_art_why_nft_conventional art
2,1,12,1_minting_opensea_on opensea_in mintableapp
3,2,6,2_gas fees_fees_fee_ethereum


In [220]:
from collections import defaultdict

docs_by_topics = defaultdict(list)

for index, topic in enumerate(topics):
  docs_by_topics[topic].append({
      'docIdx': index,
      'prob': probabilities[index][topic+1]
  })

# sort doc by their topic prob
for index, topic in enumerate(topics):
  docs_by_topics[topic].sort(key=lambda x: x['prob'], reverse=True)


In [222]:
for doc in docs_by_topics[2][:20]:
  print(doc['prob'], df_selection['title'][doc['docIdx']])


0.0 Why are the Ethereum gas fees so high? I have to pay double for the price of the NFT I want to buy.
0.0 Why would I choose Traditional and pass gas fees?
0.0 New to NFT. Why my gas fee is so high?
0.0 Why are the Ethereum gas fees so high? I have to pay double for the price of the NFT I want to buy.
0.0 Why would I choose Traditional and pass gas fees?
0.0 New to NFT. Why my gas fee is so high?


In [71]:
topics_over_time = topic_model.topics_over_time(df_selection['title'], topics, df_selection['created_utc'], nr_bins=20)
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

In [72]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_heatmap()

In [ ]:
topic_model.visualize_hierarchy()